OK!今天會做這個題目主要是有感於資管專題下禮拜有說明會，然後也差不多要開始找教授了。
雖然感覺教授們會比較關注技術面(跟我一樣)但是如果用到某個商業實踐，我們有哪些領域可以考慮呢？
我第一個會想分析一下App的市場，看一下成功的App們有什麼特質。畢竟我們現在實際有在用的App幾乎都是社群軟體，會影響對好App的評價。
如果主觀就挑幾款App分析顯得不夠客觀，因此既然有找到Google的App資料，就直接來分析一下吧！

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/googleplaystore.csv')
df.head()

In [ ]:
"""
data_overview(df)
input : df : DataFrame
output:簡單的資料概覽
"""
def data_overview(df):
    print("Data columns :" , df.shape[1])
    print("Data nums : " , df.shape[0])
    print(df.info())
    print(df.head())
data_overview(df)

好吧，分數缺失的比較多，我們可以把題目改成作剩下的分數預測XD
不過因為今天的目的是：「分析App的定價跟成功的App有什麼秘訣？」
因此我們先按捺住那股衝動，做個EDA!
在EDA之前不要忘記做個資料清洗，觀察一下可以發現size有M，我們把MB去掉保留數字就好！很多App後面的下載數用＋表示，那個我們也想拿掉。
其實原則很簡單，就是把數字還給是數字的樣子！有個要注意的點是裡面有個叫Free的名稱，那應該是不小心記到price，可以看看資料是否互換了！

# Data cleasing

In [ ]:
cleasing_name = ['Rating' ,'Reviews' ,'Size' , 'Installs', 'Price'
,'Current Ver', 'Android Ver']
for col in cleasing_name:
    print(df[col].unique())

ok，這樣我們可以很快概覽這些資料的長相(?
那就知道分別對應的col有哪些資料需要結構化，比如k可以改成1000，and up 可以拿掉，反正就是把奇奇怪怪的符號轉化一下！
有意義的給他數字意義，沒意義的了解意思後思考替換方式，然後維持數字的樣子

In [ ]:
df[df.Installs =="Free"] #看一下，發現原來他沒有被歸類到導致資料遞移，那我們就先不要看他了，因為他評分好低，方便一下刪掉

In [ ]:
df = df[~df['Installs'].isin(["Free"])]
df['Installs'] = df['Installs'].apply(lambda x: x.replace('+' ,'' ))
df['Installs'] = df['Installs'].apply(lambda x: x.replace(',' , ''))
df.Installs = df.Installs.apply(lambda x:int(x))

In [ ]:
#清理Size
df['Size'] = df.Size.apply(lambda x: str(x).replace('Varies with device' ,'NaN'))
df['Size'] = df.Size.apply(lambda x:str(x).replace('M' , '')) #把M去掉
df['Size'] = df.Size.apply(lambda x:str(x).replace(',' , ''))
df['Size'] = df.Size.apply(lambda x: float(str(x).replace('k',''))/1000)

#更改Type
df['Size'] = df.Size.apply(lambda x: float(x))
df['Installs'] = df.Installs.apply(lambda x:float(x))

#處理Price
df['Price'] = df['Price'].apply(lambda x:str(x).replace('$' , ''))
df['Price'] = df['Price'].apply(lambda x:float(x))

#把review的type轉一下
df['Reviews'] = df['Reviews'].apply(lambda x:int(x))

In [ ]:
#載入畫圖套件
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#我們簡單畫一下圖，看看資料分布
df_pair = df.dropna()
r = df.Rating
s = df.Size
i = df.Installs[df.Installs!=0]
re = df.Reviews[df.Reviews!=0]
t = df.Type
price = df['Price']
# sns.pairplot()
sns.pairplot(pd.DataFrame(list(zip(r,s,i,re,t,price)) , columns=['Rating','Size', 'Installs', 'Reviews', 'Type', 'Price']) , hue = 'Type' , palette='Set2')

那個我說，review , installs 的分佈實在不是很明顯，我們可以推估人家是右偏（左邊超高右邊沒東西），所以取對數試試看！

In [ ]:
#我們取對數畫圖，看看能不能更好洞察資料分布
df_pair = df.dropna()
r = df.Rating
s = df.Size
i = df.Installs[df.Installs!=0]
re = df.Reviews[df.Reviews!=0]
t = df.Type
price = df['Price']
# sns.pairplot()
sns.pairplot(pd.DataFrame(list(zip(r,s,np.log(i),np.log10(re),t,price)) , columns=['Rating','Size', 'Installs', 'Reviews', 'Type', 'Price']) , hue = 'Type' , palette='Set2')

簡單看一下，可以歸納出幾點：
* 評論數明顯付費App多一點，可能免費的人們不爽就刪掉了XD，但是付錢的App會開罵！
* 付費的App多數都有不錯的評價，可見如果想試試看有興趣的App(比如生產力、遊戲)，可以直接上網看「付費App」的評價來看一下這款App有沒有滿足到您的需求。
* 大多數人還是偏好安裝輕型App，畢竟App實在太多了，人家憑什麼要替大App留下空間，還不如多裝幾個App玩玩看
* reviews跟安裝量沒什麼關係，可見App適合敏捷開發，迅速得到市場反饋再成長即可，重點是prototype跟version的推出速度（所以我們**等等可以看看version跟review是不是有正向關係？改版越多次的拿掉日期因素之後，會不會是快速吸收市場反映的結果呢？**）
* 有些App的price也太高，這裡是美國資料，美金400真的what is this junk app?

In [ ]:
import plotly
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from plotly.offline import iplot
import plotly.figure_factory as ff
import cufflinks as cf


import warnings
warnings.filterwarnings('ignore')

In [ ]:
number_of_apps_in_category = df.Category.value_counts().sort_values()
data = [go.Pie(
    labels = number_of_apps_in_category.index,
    values = number_of_apps_in_category.values,
    hoverinfo = 'label+value'
)]

iplot(data )

In [ ]:
number_of_apps_in_category = df.Category.value_counts().sort_values(ascending = False
                                                                   )
data = [go.Bar(
            x= number_of_apps_in_category.index,
            y= number_of_apps_in_category.values,
    
    )]

iplot(data)

其實生產力工具最近幾年慢慢往上升了，人們越來越注重個人效能以及管理 <<< 這都是題外話，並不是我們分析得出的結果
回到圖表，我們發現最多的App是family()，遊戲還是熱門，的確，現在看到同學滑手機不是玩電動就是回訊息，大概前三名蠻明顯的量很多，剩下的App種類其實都差不多。
因此如果想要開創比較小的市場，可以走**中間排名**的種類App，比如運動、生活、金融、社群等等，那等一下，為什麼不是做比較冷門的種類呢？
我們可以發現：後面的App有什麼特性？主要是新興與娛樂種類，但是這些都是平台的移轉性高的App，美妝、食物、玩樂都可以用IG , FB 取代，天氣手機基本上都內建了，甚至可以
追蹤氣象專家的粉絲團關注，Art有pinterest web可以逛，這些App或許不是一個原生的，而主要都是已經有web service的App。
但是如果我們想走移動端友善的，就要思考有什麼服務可以不被網站取代，通常結合GPS、預定的都是mobile 優先的App，我們可以思考融入商務、金融等...

In [ ]:
data = [go.Histogram(
    x = df.Rating,
    xbins = {'start':1 , 'size' :0.1 , 'end':5}
)]

print('Average app rating = '  , np.mean(df.Rating))
iplot(data)

App平均都有4.19分，那麼哪些種類的App平均有比較好的表現呢？

In [ ]:
data = [go.Box(x = df.Category , y = df.Rating)]
iplot(data)

看來有很多離群值呢！有些App容易出現表現得特別差的App，但是有些App的評分就非常集中，好比social , shopping的就蠻集中的，tools , finance就蠻不集中的，尤其是
tool，難用的tool大家就會直接給1分了！或許是因為這些App的使用對象比較偏向專業人士，對資訊以及好用度非常嚴格，
值得注意的是dating , lifestyle的App雖然沒有太多離均值，但是評價偏向兩極。

# 檢查類別變數之間有沒有統計顯著性？ 評分會根據不同App有顯著不同嗎？

In [ ]:
print('Get App!')
df.Category.value_counts()

In [ ]:
import scipy.stats as stats
# 做one-way ANOVA analysis
f = stats.f_oneway(df.loc[df.Category == 'BUSINESS']['Rating'].dropna(), 
               df.loc[df.Category == 'FAMILY']['Rating'].dropna(),
               df.loc[df.Category == 'GAME']['Rating'].dropna(),
               df.loc[df.Category == 'PERSONALIZATION']['Rating'].dropna(),
               df.loc[df.Category == 'LIFESTYLE']['Rating'].dropna(),
               df.loc[df.Category == 'FINANCE']['Rating'].dropna(),
               df.loc[df.Category == 'EDUCATION']['Rating'].dropna(),
               df.loc[df.Category == 'MEDICAL']['Rating'].dropna(),
               df.loc[df.Category == 'TOOLS']['Rating'].dropna(),
               df.loc[df.Category == 'PRODUCTIVITY']['Rating'].dropna()
              )

print(f)
# 因為分析太多App種類會有點零碎，我們將前幾名抓出來就好
groups = df.groupby('Category').filter(lambda x: len(x) > 283).reset_index()

發現p-value非常小，所以我們可以拒絕虛無假設，代表評分會因為類別而有所不同

In [ ]:
#將評分資料根據類別做圖
array = groups['Rating'].hist(by = groups.Category , sharex = True , figsize = (20,20))

# 誰表現得最好？

In [ ]:
# 將video以上的App都抓出來
groups = df.groupby('Category').filter(lambda x: len(x) >=175).reset_index()

print('Average rating = ' , np.nanmean(list(groups.Rating)))

c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 720, len(set(groups.Category)))]

layout = {'title' : 'App ratings across major categories',
        'xaxis': {'tickangle':-40},
        'yaxis': {'title': 'Rating'},
          'plot_bgcolor': 'rgb(250,250,250)',
          'shapes': [{
              'type' :'line',
              'x0': -.5,
              'y0': np.nanmean(list(groups.Rating)),
              'x1': 19,
              'y1': np.nanmean(list(groups.Rating)),
              'line': { 'dash': 'dashdot'}
          }]
          }

data = [{
    'y': df.loc[df.Category==category]['Rating'], 
    'type':'violin',
    'name' : category,
    'showlegend':False,
    #'marker': {'color': 'Set2'},
    } for i,category in enumerate(list(set(groups.Category)))]


iplot({'data':data , 'layout':layout})

我們可以看到，健康健身、閱讀的App平均的品質比較高，超過半數的App分數都接近4.5分。
而約會類型的App分數有一半都小於4.18(平均分數)

# App的容量影響大嗎？

In [ ]:
sns.set_style('darkgrid')
df.Size = df.Size*100
plt.figure(figsize = (12,8))
ax = sns.jointplot(df.Size , df.Rating )

基本上App都小於20MB

In [ ]:
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, len(list(set(groups.Category))))]

#把比較重的App取出來
subset_df = df[df.Size > 40]
groups_temp = subset_df.groupby('Category').filter(lambda x: len(x) >20)

#知道group items 是家庭、工具、閱讀
# for category in enumerate(list(set(groups_temp.Category))):
#     print (category)

data = [{
    'x': groups_temp.loc[subset_df.Category==category[1]]['Rating'], 
    'type':'scatter',
    'y' : subset_df['Size'],
    'name' : str(category[1]),
    'mode' : 'markers',
    'showlegend': True,
    #'marker': {'color':c[i]}
    #'text' : df['rating'],
    } for category in enumerate(['FAMILY','TOOLS' , 'LIBRARIES_AND_DEMO'])]


layout = {'title':"Rating vs Size", 
          'xaxis': {'title' : 'Rating'},
          'yaxis' : {'title' : 'Size (in MB)'},
         'plot_bgcolor': 'rgb(0,0,0)'}

iplot({"data":data , "layout":layout})

大部分的**家庭類**、**工具類** , **閱讀類 **比較吃容量！


# 定價策略，免費的最貴？

In [ ]:
paid_apps = df[df.Price > 0]
p = sns.jointplot("Price" , "Rating" , paid_apps)

大部分的App下載是不需要付費的，但是少數幾個真的很坑爹，看一下這些App是何許人也！

In [ ]:
df[['Category' , 'App' , "Price"]][df.Price > 50].sort_values(by = "Price",ascending = False)

In [ ]:
junk_app = df[['Category' , 'App' , "Price"]][df.Price > 50]
labels = junk_app.Category.value_counts().keys().tolist()
values = junk_app.Category.value_counts().values.tolist()

data = go.Pie(labels=labels,values=values)
iplot([data])

財經、生活、家庭、醫學都偏貴，財經不知道在貴什麼（怒 ，我們接著看一看價格分佈

In [ ]:
fig , ax = plt.subplots()
fig.set_size_inches(15,8)
subset_df_price = subset_df[subset_df.Price < 100]

#可以很好看出階段分佈
p = sns.stripplot(x=  "Price" , y = "Category" , data=subset_df_price , jitter = True , linewidth=1)
title = ax.set_title('App pricing trend across categories - after filtering for expensive apps')

大部分的App還是不會超過30美金

In [ ]:
# Stacked bar graph for top 5-10 categories - Ratio of paid and free apps
#fig, ax = plt.subplots(figsize=(15,10))

new_df = df.groupby(['Category', 'Type']).agg({'App' : 'count'}).reset_index()
#print(new_df)

# outer_group_names = df['Category'].sort_values().value_counts()[:5].index
# outer_group_values = df['Category'].sort_values().value_counts()[:5].values

outer_group_names = ['GAME', 'FAMILY', 'MEDICAL', 'TOOLS']
outer_group_values = [len(df.App[df.Category == category]) for category in outer_group_names]

a, b, c, d=[plt.cm.Blues, plt.cm.Reds, plt.cm.Greens, plt.cm.Purples]


inner_group_names = ['Paid', 'Free'] * 4
inner_group_values = []
#inner_colors = ['#58a27c','#FFD433']


for category in outer_group_names:
    for t in ['Paid', 'Free']:
        x = new_df[new_df.Category == category]
        try:
            #print(x.App[x.Type == t].values[0])
            inner_group_values.append(int(x.App[x.Type == t].values[0]))
        except:
            #print(x.App[x.Type == t].values[0])
            inner_group_values.append(0)

explode = (0.025,0.025,0.025,0.025)
# First Ring (outside)
fig, ax = plt.subplots(figsize=(10,10))
ax.axis('equal')


mypie, texts, _ = ax.pie(outer_group_values, radius=1.2, labels=outer_group_names, autopct='%1.1f%%', pctdistance=1.1,
                                 labeldistance= 0.75,  explode = explode, colors=[a(0.6), b(0.6), c(0.6), d(0.6)], textprops={'fontsize': 16})
plt.setp( mypie, width=0.5, edgecolor='black')
 
# Second Ring (Inside)
mypie2, _ = ax.pie(inner_group_values, radius=1.2-0.5, labels=inner_group_names, labeldistance= 0.7, 
                   textprops={'fontsize': 12}, colors = [a(0.4), a(0.2), b(0.4), b(0.2), c(0.4), c(0.2), d(0.4), d(0.2)])
plt.setp( mypie2, width=0.5, edgecolor='black')
plt.margins(0,0)
 
# show it
plt.tight_layout()
plt.show()




# Are paid apps downloaded as much as free apps?

In [ ]:
trace0 = go.Box(
    y=np.log10(df['Installs'][df.Type=='Paid']),
    name = 'Paid',
    marker = dict(
        color = 'rgb(214, 12, 140)',
    )

)
trace1 = go.Box(
    y=np.log10(df['Installs'][df.Type=='Free']),
    name = 'Free',
    marker = dict(
        color = 'rgb(0, 128, 128)',
    )
)
layout = go.Layout(
    title = "Number of downloads of paid apps Vs free apps",
    yaxis= {'title': 'Number of downloads (log-scaled)'}
)
data = [trace0, trace1]
plotly.offline.iplot({'data': data, 'layout': layout})

付費App的下載量果然比free App還小

In [ ]:
temp_df = df[df.Type == 'Paid']
temp_df = temp_df[temp_df.Size > 5]
#type_groups = df.groupby('Type')

data = [{
    #'x': type_groups.get_group(t)['Rating'], 
    'x' : temp_df['Rating'],
    'type':'scatter',
    'y' : temp_df['Size'],
    #'name' : t,
    'mode' : 'markers',
    #'showlegend': True,
    'text' : df['Size'],
    } for t in set(temp_df.Type)]


layout = {'title':"Rating vs Size", 
          'xaxis': {'title' : 'Rating'},
          'yaxis' : {'title' : 'Size (in MB)'},
         'plot_bgcolor': 'rgb(0,0,0)'}

plotly.offline.iplot({'data': data, 'layout': layout})

# 使用者會因為評論數比較多而下載App嗎？

In [ ]:
#df['Installs'].corr(df['Reviews'])#df['Insta 
#print(np.corrcoef(l, rating))

corrmat = df.corr()
#f, ax = plt.subplots()
p =sns.heatmap(corrmat, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))

In [ ]:
df_copy = df.copy()

df_copy = df_copy[df_copy.Reviews > 10]
df_copy = df_copy[df_copy.Installs > 0]

df_copy['Installs'] = np.log10(df['Installs'])
df_copy['Reviews'] = np.log10(df['Reviews'])

sns.lmplot("Reviews", "Installs", data=df_copy)
ax = plt.gca()
_ = ax.set_title('Number of Reviews Vs Number of Downloads (Log scaled)')

我們可以看到有個很明顯的正向關係，代表使用者偏好評論數多的App，但也有可能是下載量大，所以給予反饋的評論比較多。說到評論，我們就來分析一下情緒

In [ ]:
reviews_df = pd.read_csv('../input/googleplaystore_user_reviews.csv')
reviews_df.head()

In [ ]:
#跟原本的表作交集
merged_df = pd.merge(df, reviews_df, on = "App", how = "inner")
merged_df = merged_df.dropna(subset=['Sentiment', 'Translated_Review'])

In [ ]:
grouped_sentiment_category_count = merged_df.groupby(['Category', 'Sentiment']).agg({'App': 'count'}).reset_index()
grouped_sentiment_category_sum = merged_df.groupby(['Category']).agg({'Sentiment': 'count'}).reset_index()

new_df = pd.merge(grouped_sentiment_category_count, grouped_sentiment_category_sum, on=["Category"])
#print(new_df)
new_df['Sentiment_Normalized'] = new_df.App/new_df.Sentiment_y
new_df = new_df.groupby('Category').filter(lambda x: len(x) ==3)
# new_df = new_df[new_df.Category.isin(['HEALTH_AND_FITNESS', 'GAME', 'FAMILY', 'EDUCATION', 'COMMUNICATION', 
#                                      'ENTERTAINMENT', 'TOOLS', 'SOCIAL', 'TRAVEL_AND_LOCAL'])]
new_df

trace1 = go.Bar(
    x=list(new_df.Category[::3])[6:-5],
    y= new_df.Sentiment_Normalized[::3][6:-5],
    name='Negative',
    marker=dict(color = 'rgb(209,49,20)')
)

trace2 = go.Bar(
    x=list(new_df.Category[::3])[6:-5],
    y= new_df.Sentiment_Normalized[1::3][6:-5],
    name='Neutral',
    marker=dict(color = 'rgb(49,130,189)')
)

trace3 = go.Bar(
    x=list(new_df.Category[::3])[6:-5],
    y= new_df.Sentiment_Normalized[2::3][6:-5],
    name='Positive',
    marker=dict(color = 'rgb(49,189,120)')
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    title = 'Sentiment analysis',
    barmode='stack',
    xaxis = {'tickangle': -45},
    yaxis = {'title': 'Fraction of reviews'}
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot({'data': data, 'layout': layout})

健身還有活動類型的App正向評論比例非常高。
有趣的是，遊戲類App很少中立評價，可能手遊玩家都是對一個遊戲「有心得之後」就會馬上來評分。
新聞類App的負評偏多，可能跟人們討厭無用的報導有關

In [ ]:
sns.set_style('ticks')
sns.set_style('darkgrid')
fig , ax = plt.subplots()
fig.figsize = (12 , 8)
ax = sns.boxplot(x = 'Type' , y = 'Sentiment_Polarity' , data=merged_df)
title = ax.set_title('Sentiment Polarity Distribution')


非常有趣的是，免費App大家都罵得蠻用力的，很多負分（免費的你還嫌XD
但是付費App就少非常多負向分數！

# 他們都在說啥？

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(background_color="white", max_words=200, colormap="Set2")
# generate word cloud

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop = stop + ['app', 'APP' ,'ap', 'App', 'apps', 'application', 'browser', 'website', 'websites', 'chrome', 'click', 'web', 'ip', 'address',
            'files', 'android', 'browse', 'service', 'use', 'one', 'download', 'email', 'Launcher']

#merged_df = merged_df.dropna(subset=['Translated_Review'])
merged_df['Translated_Review'] = merged_df['Translated_Review'].apply(lambda x: " ".join(x for x in str(x).split(' ') if x not in stop))
#print(any(merged_df.Translated_Review.isna()))
merged_df.Translated_Review = merged_df.Translated_Review.apply(lambda x: x if 'app' not in x.split(' ') else np.nan)
merged_df.dropna(subset=['Translated_Review'], inplace=True)


free = merged_df.loc[merged_df.Type=='Free']['Translated_Review'].apply(lambda x: '' if x=='nan' else x)
wc.generate(''.join(str(free)))
plt.figure(figsize=(10, 10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
paid = merged_df.loc[merged_df.Type=='Paid']['Translated_Review'].apply(lambda x: '' if x=='nan' else x)
wc.generate(''.join(str(paid)))
plt.figure(figsize=(10, 10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

# 結論：

* 平均的App分數大概是4.18
* 大部分的App都在2MB左右
* 定價基本上都在30美金以內
* 財經、生活、家庭、醫學的App有一些坑錢的
* 健身App品質算是蠻穩定的（也可能他的使用者偏好給高分（？）約會、遊戲就比較普通
* 免費App的營運商，不能太玻璃心XD